# About the notebook

In this notebook, the dataframes representing the relationships between different combinations of fields from the notebook __`G_Final_2_Graph_Relationship_Data.ipynb`__ are used to recommend songs based on network graph theory and an appropriate shortest-path algorithm. The `networkx` library will be used for the graph analysis and `pyplot` module of `matplotlib` library will be used for the visualization.

### Import the libraries and set random seed 

In [1]:
#%matplotlib inline
import os
import re
import itertools as it
import pandas as pd
import numpy as np
import operator 
import functools
import pickle as pk
import networkx as nx
import matplotlib as mplot
import matplotlib.colors as colors
import matplotlib.pyplot as pyplot
import random 

In [4]:
# Set the seed for the random number generator - to be consistent in different notebooks
a = random.randint(0,9)
random.seed(4)
a

3

### Set the desired plot style for `pyplot`

In [2]:
# Change the styles of the pyplot plots
print(pyplot.style.available) # shows available styles
pyplot.style.use('classic')
pyplot.rcParams['figure.figsize'] = (10.0, 8.0)

['seaborn-pastel', 'bmh', 'seaborn-dark-palette', 'seaborn-notebook', 'ggplot', 'seaborn-talk', 'seaborn-ticks', 'dark_background', 'seaborn-whitegrid', 'seaborn-bright', 'fivethirtyeight', 'classic', 'seaborn-colorblind', 'seaborn-darkgrid', 'seaborn-poster', 'seaborn-white', 'seaborn-paper', 'seaborn-muted', 'seaborn-dark', 'seaborn-deep', 'grayscale']


## Weighted Network Graph 1: represent all node types

In the first network graph, all the three node types `track`, `artist_id` and `artist_terms` are represented using different colors. Also, the weights between an `artist_id` node and `artist_terms` node are computed from the group aggregates. The `networkx` and the `matplotlib` libraries will be used in conjunction with the `pandas` library for this analysis.

### Read the pickled data frames that hold the relationships and nodes

In [10]:
path = "C:/Users/ganes_000/Documents/MSBA/Spring 2016/MA 755/Pickle Files"
# Get the track-artist relationship dataframe
df_ta = pd.read_pickle(path+"/MSS_METADATA_SUBSET_500_TRACK_ARTIST.pkl")

# Get the artist-artist_terms relationship dataframe
df_aat = pd.read_pickle(path+"/MSS_METADATA_SUBSET_500_ARTIST_ARTIST_TERMS.pkl")

# Get the artist-artist relationship dataframe
df_aa = pd.read_pickle(path+"/MSS_METADATA_SUBSET_500_ARTIST_ARTIST.pkl")

In [11]:
print('TRACK-ARTIST Relationship \n ======================== \n',len(df_ta),'rows \n ----------------------\n',
      df_ta.columns)
print('\n ######################################## \n')
print('ARTIST-ARTIST TERMS Relationship \n ======================== \n',len(df_aat),'rows \n ----------------------\n',
      df_aat.columns)
print('\n ######################################## \n')
print('ARTIST-ARTIST Relationship \n ======================== \n', len(df_aa), 'rows \n ----------------------\n',
      df_aa.columns)

TRACK-ARTIST Relationship 
 500 rows 
 ----------------------
 Index(['track', 'artist_id', 'title', 'artist_name', 'artist_familiarity',
       'artist_hotttnesss', 'song_hotttnesss'],
      dtype='object')

 ######################################## 

ARTIST-ARTIST TERMS Relationship 
 12575 rows 
 ----------------------
 Index(['artist_id', 'artist_terms', 'artist_name', 'artist_familiarity',
       'artist_hotttnesss', 'at_wt', 'at_freq'],
      dtype='object')

 ######################################## 

ARTIST-ARTIST Relationship 
 198774 rows 
 ----------------------
 Index(['artist_id_l', 'artist_id_r', 'artist_name_l', 'artist_name_r',
       'wt_l_sum', 'wt_r_sum', 'fam_l', 'fam_r', 'hot_l', 'hot_r'],
      dtype='object')


### Create the Nodes - with attributes

An empty undirected graph is created and then the nodes and edges are added. Refer to the comments for greater details.

In [54]:
# Create an empty graph
G_1 = nx.Graph()

# Add nodes
G_1.add_nodes_from(df_ta['track'], type = 'track', color = 'red', hot = 0, name = 'NaN')
G_1.add_nodes_from(df_ta['artist_id'], type = 'artist', color = 'deepskyblue', hot = 0, name = 'NaN')
G_1.add_nodes_from(df_aat['artist_terms'], type = 'artist_terms', color = 'greenyellow', name = 'NaN')

print(type(G_1.nodes()))
print('No. of nodes:', G_1.number_of_nodes())

<class 'list'>
No. of nodes: 2530


In [55]:
# Get the number of nodes per node type
node_type_dict = {}

for node,data in G_1.nodes_iter(data = True):
    if data['type'] not in node_type_dict:
        node_type_dict[data['type']] = 1
    else:
        node_type_dict[data['type']] +=1
             
print(node_type_dict)

{'track': 500, 'artist': 469, 'artist_terms': 1561}


So, there are __500__ `track` nodes, __469__ `artist_id` nodes and __1561__ `artist_terms` nodes to total up to __2530__ nodes.

In [56]:
print(type(G_1.nodes())) # list of dictionaries
print(type(G_1.node['new wave']))
print(G_1.node['new wave'])
print(G_1.node['new wave']['type'])

<class 'list'>
<class 'dict'>
{'type': 'artist_terms', 'name': 'NaN', 'color': 'greenyellow'}
artist_terms


#### Add node attributes

The different attributes are added for each node type as described below.

In [57]:
# Include song and artist hotttnesss as attributes for each node
for node,data in G_1.nodes_iter(data = True):
    if data['type'] == 'track':
        data['hot'] = df_ta[df_ta['track'] == node]['song_hotttnesss'].values[0] # if [0] is missed, it will be an array (cell)
        data['name'] = df_ta[df_ta['track'] == node]['title'].values[0] 
    else:
        if data['type'] == 'artist' :
            data['hot'] = df_ta[df_ta['artist_id'] == node]['artist_hotttnesss'].values[0]
            data['name'] = df_ta[df_ta['artist_id'] == node]['artist_name'].values[0]
        else:
            data['name'] = df_aat[df_aat['artist_terms'] == node]['artist_terms'].values[0]
    
G_1.nodes(data = True)


[('AR2DKN01187FB3AA39',
  {'color': 'deepskyblue',
   'hot': 0.11241786757300369,
   'name': 'Wesley Tuttle',
   'type': 'artist'}),
 ('TRASNTF128F426C3E6',
  {'color': 'red',
   'hot': 0.59477944466256449,
   'name': 'Careless Talk',
   'type': 'track'}),
 ('harp', {'color': 'greenyellow', 'name': 'harp', 'type': 'artist_terms'}),
 ('boogie',
  {'color': 'greenyellow', 'name': 'boogie', 'type': 'artist_terms'}),
 ('maryland',
  {'color': 'greenyellow', 'name': 'maryland', 'type': 'artist_terms'}),
 ('ARYQ8281187FB58C34',
  {'color': 'deepskyblue',
   'hot': 0.38866990102322235,
   'name': 'Royal S',
   'type': 'artist'}),
 ('77 style punk',
  {'color': 'greenyellow', 'name': '77 style punk', 'type': 'artist_terms'}),
 ('drama', {'color': 'greenyellow', 'name': 'drama', 'type': 'artist_terms'}),
 ('modern classical',
  {'color': 'greenyellow',
   'name': 'modern classical',
   'type': 'artist_terms'}),
 ('ARUSOMY1187FB4D3B4',
  {'color': 'deepskyblue',
   'hot': 0.52437898751480838,
  

### Add the Edges - with attributes and weight

There will be two types of edges
+ `track` to `artist_id`
+ `artist_id` to `artist_terms`

First, the `track` to `artist_id` edges are added.

In [58]:
# using from pandas dataframe ()
# create a weight column in the df_ta dataframe and set it as 1. This will be the weight.
df_ta1 = df_ta
df_ta1['weight'] = 1
# track - artist edges
graph_edge_1_pd = nx.from_pandas_dataframe(df_ta1, 'track', 'artist_id', 
                                           edge_attr=['weight','song_hotttnesss','artist_hotttnesss','artist_familiarity'], 
                                           create_using=nx.Graph())

print(type(graph_edge_1_pd))

# create an edge list
edges_1 = nx.to_edgelist(graph_edge_1_pd)
print(type(edges_1))

# add edges to graph
G_1.add_edges_from(edges_1)

print('No. of track to artist edges:', G_1.number_of_edges())


<class 'networkx.classes.graph.Graph'>
<class 'list'>
No. of track to artist edges: 500


There are 500 `track` to `artist_id` edges. Now, the `artist_id` to `artist_terms` edges are added.

In [59]:
# rename at_wt column as weight in the df_aat dataframe. This i to ensure consistency of weight attribute for all edge types.
df_aat1 = df_aat
df_aat1['weight'] = df_aat['at_wt']

# artist - artist_terms edges
graph_edge_2_pd = nx.from_pandas_dataframe(df_aat1, 'artist_terms', 'artist_id', 
                                           edge_attr=['weight','artist_hotttnesss','artist_familiarity'], 
                                           create_using=nx.Graph())

print(type(graph_edge_2_pd))
edges_2 = nx.to_edgelist(graph_edge_2_pd)
print(type(edges_2))

# add edges to graph
G_1.add_edges_from(edges_2)

print('No. of artist - artist_terms edges:', G_1.number_of_edges()-500)

<class 'networkx.classes.graph.Graph'>
<class 'list'>
No. of artist - artist_terms edges: 12428


In [44]:
# Look at each of the edge lists with data
G_1.edges(data = True)

[('AR2DKN01187FB3AA39',
  'guitar',
  {'artist_familiarity': 0.3004990465562955,
   'artist_hotttnesss': 0.11241786757300369,
   'weight': 0.36152517991705674}),
 ('AR2DKN01187FB3AA39',
  'traditional country',
  {'artist_familiarity': 0.3004990465562955,
   'artist_hotttnesss': 0.11241786757300369,
   'weight': 0.6655481371159804}),
 ('AR2DKN01187FB3AA39',
  'country',
  {'artist_familiarity': 0.3004990465562955,
   'artist_hotttnesss': 0.11241786757300369,
   'weight': 0.5507469775270242}),
 ('AR2DKN01187FB3AA39',
  'cowboy',
  {'artist_familiarity': 0.3004990465562955,
   'artist_hotttnesss': 0.11241786757300369,
   'weight': 0.6309211941647311}),
 ('AR2DKN01187FB3AA39',
  'western swing',
  {'artist_familiarity': 0.3004990465562955,
   'artist_hotttnesss': 0.11241786757300369,
   'weight': 1.0}),
 ('AR2DKN01187FB3AA39',
  'TRAMFIU128F426B004',
  {'artist_familiarity': 0.3004990465562955,
   'artist_hotttnesss': 0.11241786757300369,
   'song_hotttnesss': 0.26586104921065007,
   'wei

Notice the different attributes for the two different edge types in the above output. 

### Draw the weighted network graph

The weights will be used only for the `artist_id` and `artist_terms` connection. The `at_wt`, which are normalized across each `artist_id` will be used as the weights for the edges. For the `track` - `artist_id` edges, a standard weight of 1 is used. The weights will be represented by the `width` of the edges in the graph. The thicker the edge, the stronger the relationship. Also, a `spring_layout` is used which will ensure that the nodes are distributed so as to create an equilibrium state. In simpler terms, more connected (weight based) nodes will be at the centre while the lesser connected ones will be farther from the centre.

In [61]:
nx.draw(G_1,
        pos=nx.spring_layout(G_1), # Or spectral_layout, random_layout, shell_layout , spring_layout     
        with_labels = False, # True will have labels
        node_size=50, 
        alpha=0.8, 
        node_color = [G_1.node[node]['color'] for node in G_1],#node_color_values['color'],
        font_size = 7,
        font_color = 'black',
        font_family = 'arial',
        width = [edge[2]['weight'] for edge in G_1.edges(data = True)],
        edge_color = 'gray')

pyplot.show()

As can be seen from the network graph, the `artist_id` nodes (in sky blue) and many `artist_terms` nodes (in green yellow) are the most central in this weighted graph. This is as expected since the `artist_id` along with the `artist_terms` primarily define relationships and also have a higher degree, on an average, than the `track` nodes (in red) which have a degree of 1 (connected to a single artist with weight 1). The less important `artist_terms` and a handful of `artist_id` that are farther, from the centre, than the `track` nodes are less important to this network graph of the 500 songs subset of the MSS data.

In [64]:
print('The density of the graph: ',nx.density(G_1))

The density of the graph:  0.00404102919962428


However, this graph, although weighted, is not easy to work with for recommendation purposes. This is because of the three different node types that requires the modification of the inbuilt shortest-distance algorithms in `networkx`. After some failed attempts, it made sense to use the `track` - `track` relationship dataframe to create a weighted `track` - `track` network graph.  

## Weighted Network Graph 2: only `track` - `track` relationship

In the second network graph, only the `track` node will be represented. The weights between two `track` nodes are computed from the group aggregates of the `track` - `track` relationship data created in the notebook __`G_Final_2_Graph_Relationship_Data.ipynb`__ , using the rationale below. 

### The math behind the weights between two `track` nodes

We describe the math based on two case examples, first.

+ A `track` node is always connected to an `artist_id` node. 
+ Following which two `artist_id` nodes will be connected, if at all they are connected, via `n` `artist_terms` node. In other words,there will be `n` paths between the two artists (and also between the two tracks). 
+ The next step is to logically define a mathematical formula to compute the weights between the two `track` nodes.
    - Since the artist term weights are normalized for every artist, the weights will be different for the two paths reaching an `artist_terms` node from the two `artist_id` nodes. Considering that this is a path between two artists, we can add them up to compute the weight. 
    - But, there are two other attributes pertinent to each artist that could help make the weights even more effective for recommendation purposes. These are the `artist_hotttnesss` and `artist_familiarity`. These two attributes are normalized across all artists.
    - Therefore, the most logical way to define the weight of a path between an `artist_id` node and an `artist_terms` node, that is reasonably normalized across the entire graph, would be to multiply the artist term weight (`at_wt`) by a factor that is representative of the normalized recognizability of an artist. This factor is defined to be the sum of the `artist_hotttnesss` and `artist_familiarity`. 
    - Thus, the mathematical formula for the weights between two `artist_id` nodes, and hence two `track` nodes, that are connected by `n` common terms, will be given by the formula below.
    
sum[(at_wt_l)(i)*(familiarity_l + hotness_l)(i) + (at_wt_r)(i)*(familiarity_r + hotness_r)(i)] for i from 1 to n (for n common artist terms)

which reduces to 

(familiarity_l + hotness_l)*sum[(at_wt_l)(i)] + (familiarity_r + hotness_r)*sum[(at_wt_r)(i)] for i from 1 to n, where the subscripts _r and _l denote the path to the artist_term from either direction.

This term sum[(at_wt_l)(i)] is nothing but the aggregated sum of each `track` - `track` group weights, which we already have computed.


+ The key point to note here is that, the formula, in general, provides a higher weight if the two `artist_id` nodes that connect the two `track` nodes is the same. This would be true if the artist enjoys a reasonably high familarity and hotness. This means that, for recommendation, songs performed by the same artist has a higher probability of being recommended. However, in the event that the artist has a pretty poor rating, then there can be other artists with a much improved rating and connected to this artist through the artist terms, whose songs might be recommended.


### Read the pickled data frames that hold the relationships and nodes

In [70]:
path = "C:/Users/ganes_000/Documents/MSBA/Spring 2016/MA 755/Pickle Files"
df_ss = pd.read_pickle(path+"/MSS_METADATA_SUBSET_500_TRACK_TRACK.pkl")

In [71]:
print('TRACK-TRACK Relationship \n ======================== \n',len(df_ss),'rows \n ----------------------\n',
      df_ss.columns)
print('\n ######################################## \n')

TRACK-TRACK Relationship 
 198840 rows 
 ----------------------
 Index(['track_l', 'track_r', 'wt_l_sum', 'wt_r_sum', 'fam_l', 'fam_r', 'hot_l',
       'hot_r', 'song_hot_l', 'song_hot_r', 'cnt_at', 'title_l', 'artist_id_l',
       'artist_name_l', 'title_r', 'artist_id_r', 'artist_name_r',
       'same_artist_flag'],
      dtype='object')

 ######################################## 



In [72]:
df_ss.head(10)

,track_l,track_r,wt_l_sum,wt_r_sum,fam_l,fam_r,hot_l,hot_r,song_hot_l,song_hot_r,cnt_at,title_l,artist_id_l,artist_name_l,title_r,artist_id_r,artist_name_r,same_artist_flag
0,TRAAAVG12903CFA543,TRAACTB12903CAAF15,0.734100,0.724350,0.550514,0.523715,0.422706,0.384611,NaN,0.0,1,Insatiable (Instrumental Version),ARNTLGG11E2835DDB9,Clp,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0
1,TRAABJV128F1460C49,TRAACTB12903CAAF15,1.804455,2.608040,0.776676,0.523715,0.553072,0.384611,NaN,0.0,4,Tonight Will Be Alright,ARIK43K1187B9AE54C,Lionel Richie,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0
2,TRAADLN128F14832E9,TRAACTB12903CAAF15,3.067252,4.877071,0.814830,0.523715,0.735766,0.384611,0.834493,0.0,7,Angie (1993 Digital Remaster),ARFCUN31187B9AD578,The Rolling Stones,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0
3,TRAADLR12903CF8D7E,TRAACTB12903CAAF15,1.949940,2.312114,0.482356,0.523715,0.350079,0.384611,0.000000,0.0,3,Sabor Guajiro,AR6YDKV1187B989230,Roberto Torres,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0
4,TRAAENP128F147BF32,TRAACTB12903CAAF15,1.250377,2.040589,0.749318,0.523715,0.481656,0.384611,0.226555,0.0,3,Movement 4 [from Kiss] (Album Version),ARUI8651187B9ACF52,John Cale,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0
5,TRAAGHM128EF35CF8E,TRAACTB12903CAAF15,3.283752,3.739735,0.408389,0.523715,0.139247,0.384611,0.198785,0.0,5,Next Time,ARMBYRO1187FB57419,Nadine Renee,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0
6,TRAAGOH128F42593CE,TRAACTB12903CAAF15,4.317186,4.545478,0.449324,0.523715,0.343282,0.384611,0.382723,0.0,6,You Feel Good All Over,ARLWR721187B9A03C9,T.G. Sheppard,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0
7,TRAAGZU128EF360F47,TRAACTB12903CAAF15,0.706473,0.691292,0.420546,0.523715,0.313569,0.384611,NaN,0.0,1,Caricia Y Herida,ARHFEBO11F50C4825A,Flor Silvestre,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0
8,TRAAHEH128F427FCEF,TRAACTB12903CAAF15,2.292019,2.194105,0.138188,0.523715,0.317811,0.384611,NaN,0.0,3,Ter\xc3\xa4slintu,ARL6XZC1187FB3936E,Solistiyhtye Suomi,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0
9,TRAAJJG128F4284B27,TRAACTB12903CAAF15,5.544593,5.923369,0.703349,0.523715,0.498395,0.384611,0.466305,0.0,8,Medicate Myself,ARQOC971187B9910FA,The Verve Pipe,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0


### Compute the weights and inverse weights for the `track` - `track` relationship

The formula discussed above is used to compute the weights. Although weights relate directly to the strength of the relationship and it is necessary for visualization purposes, for shortest-distance algorithms the distance between nodes must be represented as inverse weights. 

In [73]:
# Calculate the weights for each connection
df_ss['weight'] = df_ss['wt_l_sum']*(df_ss['fam_l'] + df_ss['hot_l']) + df_ss['wt_r_sum']*(df_ss['fam_r'] + df_ss['hot_r'])

# Inverse weights
df_ss['inv_weight'] = 1/df_ss['weight']

df_ss.head(10)

,track_l,track_r,wt_l_sum,wt_r_sum,fam_l,fam_r,hot_l,hot_r,song_hot_l,song_hot_r,cnt_at,title_l,artist_id_l,artist_name_l,title_r,artist_id_r,artist_name_r,same_artist_flag,weight,inv_weight
0,TRAAAVG12903CFA543,TRAACTB12903CAAF15,0.734100,0.724350,0.550514,0.523715,0.422706,0.384611,NaN,0.0,1,Insatiable (Instrumental Version),ARNTLGG11E2835DDB9,Clp,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0,1.372386,0.728658
1,TRAABJV128F1460C49,TRAACTB12903CAAF15,1.804455,2.608040,0.776676,0.523715,0.553072,0.384611,NaN,0.0,4,Tonight Will Be Alright,ARIK43K1187B9AE54C,Lionel Richie,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0,4.768420,0.209713
2,TRAADLN128F14832E9,TRAACTB12903CAAF15,3.067252,4.877071,0.814830,0.523715,0.735766,0.384611,0.834493,0.0,7,Angie (1993 Digital Remaster),ARFCUN31187B9AD578,The Rolling Stones,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0,9.186036,0.108861
3,TRAADLR12903CF8D7E,TRAACTB12903CAAF15,1.949940,2.312114,0.482356,0.523715,0.350079,0.384611,0.000000,0.0,3,Sabor Guajiro,AR6YDKV1187B989230,Roberto Torres,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0,3.723351,0.268575
4,TRAAENP128F147BF32,TRAACTB12903CAAF15,1.250377,2.040589,0.749318,0.523715,0.481656,0.384611,0.226555,0.0,3,Movement 4 [from Kiss] (Album Version),ARUI8651187B9ACF52,John Cale,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0,3.392701,0.294750
5,TRAAGHM128EF35CF8E,TRAACTB12903CAAF15,3.283752,3.739735,0.408389,0.523715,0.139247,0.384611,0.198785,0.0,5,Next Time,ARMBYRO1187FB57419,Nadine Renee,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0,5.195200,0.192485
6,TRAAGOH128F42593CE,TRAACTB12903CAAF15,4.317186,4.545478,0.449324,0.523715,0.343282,0.384611,0.382723,0.0,6,You Feel Good All Over,ARLWR721187B9A03C9,T.G. Sheppard,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0,7.550600,0.132440
7,TRAAGZU128EF360F47,TRAACTB12903CAAF15,0.706473,0.691292,0.420546,0.523715,0.313569,0.384611,NaN,0.0,1,Caricia Y Herida,ARHFEBO11F50C4825A,Flor Silvestre,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0,1.146552,0.872181
8,TRAAHEH128F427FCEF,TRAACTB12903CAAF15,2.292019,2.194105,0.138188,0.523715,0.317811,0.384611,NaN,0.0,3,Ter\xc3\xa4slintu,ARL6XZC1187FB3936E,Solistiyhtye Suomi,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0,3.038119,0.329151
9,TRAAJJG128F4284B27,TRAACTB12903CAAF15,5.544593,5.923369,0.703349,0.523715,0.498395,0.384611,0.466305,0.0,8,Medicate Myself,ARQOC971187B9910FA,The Verve Pipe,It Makes No Difference Now,AR0RCMP1187FB3F427,Billie Jo Spears,0,12.043534,0.083032


### Create the Nodes - with attributes

An empty undirected graph is created and then the nodes and edges are added. The track title will be used as the key for the graph, as opposed to the track id, purely for better visualization. Refer to the comments for greater details.

In [74]:
# Create an empty graph
G_2 = nx.Graph()

# Add nodes
# Using the title as the node key instead of track
G_2.add_nodes_from(df_ss['title_l'], type = 'track', color = 'crimson',s_hot = 0, track_id = 'NaN', 
                   artist_name = 'NaN', artist_id = 'NaN', a_hot = 0, a_fam = 0)
G_2.add_nodes_from(df_ss['title_r'], type = 'track', color = 'crimson',s_hot = 0, track_id = 'NaN', 
                   artist_name = 'NaN', artist_id = 'NaN', a_hot = 0, a_fam = 0)


print(type(G_2.nodes()))
print('No. of nodes:', G_2.number_of_nodes())

<class 'list'>
No. of nodes: 500


#### Add node attributes

The different attributes are added for the `track` node as described below.

In [75]:
for node,data in G_2.nodes_iter(data = True):
    if data['type'] == 'track':
        data['s_hot'] = df_ss[df_ss['title_l'] == node]['song_hot_l'].values[0] # if [0] is missed, it will be an array (cell)
        data['track_id'] = df_ss[df_ss['title_l'] == node]['track_l'].values[0] 
        data['artist_name'] = df_ss[df_ss['title_l'] == node]['artist_name_l'].values[0]
        data['artist_id'] = df_ss[df_ss['title_l'] == node]['artist_id_l'].values[0]
        data['a_hot'] = df_ss[df_ss['title_l'] == node]['hot_l'].values[0]
        data['a_fam'] = df_ss[df_ss['title_l'] == node]['fam_l'].values[0]
        
G_2.nodes(data = True)

[('Allein',
  {'a_fam': 0.47012407514783316,
   'a_hot': 0.32502293284522349,
   'artist_id': 'ARQAXT91187B99103A',
   'artist_name': 'UKW',
   'color': 'crimson',
   's_hot': 0.0,
   'track_id': 'TRABWUW128F93214D6',
   'type': 'track'}),
 ('Missing You',
  {'a_fam': 0.53444360241472444,
   'a_hot': 0.33358462783523912,
   'artist_id': 'AR52QY11187B99FC59',
   'artist_name': 'Anthony Cruz',
   'color': 'crimson',
   's_hot': 0.26586104921065007,
   'track_id': 'TRAVOFJ128F933DEA1',
   'type': 'track'}),
 ("Class Compromise (&History's Glory)",
  {'a_fam': 0.62516576239137323,
   'a_hot': 0.4427392788160599,
   'artist_id': 'ARRXADQ1187FB38F22',
   'artist_name': 'The Business',
   'color': 'crimson',
   's_hot': 0.36037058271214062,
   'track_id': 'TRAZISB128F427907A',
   'type': 'track'}),
 ('Kiss the Sky',
  {'a_fam': 0.50526939552243411,
   'a_hot': 0.0,
   'artist_id': 'ARORFIN1187FB3BE5C',
   'artist_name': 'Danielle Bollinger',
   'color': 'crimson',
   's_hot': nan,
   'track_i

### Add the Edges - with attributes and weight

First, the `track` to `track` edges are added, with the computed weights and the inverse weights along with the other attributes.

In [76]:
# Adding the Edges
# using from pandas dataframe ()
graph_edge_pd = nx.from_pandas_dataframe(df_ss, 'title_l', 'title_r', 
                                           edge_attr=['artist_name_l','artist_name_r',
                                                      'hot_l','hot_r','fam_l','fam_r',
                                                      'wt_l_sum','wt_r_sum',
                                                      'cnt_at','weight','inv_weight'], 
                                           create_using=nx.Graph())
print(type(graph_edge_pd))
edges = nx.to_edgelist(graph_edge_pd)

print(type(edges))
G_2.add_edges_from(edges)

print('No. of track to track edges:', G_2.number_of_edges())

<class 'networkx.classes.graph.Graph'>
<class 'list'>
No. of track to track edges: 99420


The `df_ss` dataframe contained __198,840__ rows indicating that there are those many number of direct connections between `track` - `track` nodes. But since there are reverse paths too, it was pointed to earlier that these will be ignored by `networkx` and that the number of edges will be halved. Note that __99,420__ edges above. It would have been halved even if a `MultiGraph` was used since the edge attributes are the same for both the forward and reverse paths.

### Draw the weighted network graph

The computed weights will be used as the weights for the edges. The weights will be represented by the `width` of the edges in the graph. The thicker the edge, the stronger the relationship. To reduce the effect, a multiplicative factor of 0.25 is used, owing to the larger absolute weights. Also, a `spring_layout` is used which will ensure that the nodes are distributed so as to create an equilibrium state. In simpler terms, more connected (weight based) nodes will be at the centre while the lesser connected ones will be farther from the centre.

In [79]:
# Draw the network graph
nx.draw(G_2,
        pos=nx.spring_layout(G_2), # Or spectral_layout, random_layout, shell_layout , spring_layout     
        with_labels = False,
        node_size=100, 
        alpha=0.8, 
        node_color = [G_2.node[node]['color'] for node in G_2],#node_color_values['color'],
        font_size = 7,
        font_color = 'black',
        font_family = 'arial',
        edge_color = 'gray',
        width = [0.25*edge[2]['weight'] for edge in G_2.edges(data=True)])
        
pyplot.show()

As can be seen from the network graph, almost all the `track` nodes form one big cluster. This is probably because, all the connections between `track` nodes have been considered. In other words, all ranges of weights have been included as a valid relationship. Maybe if edges that have weights below a threshold are dropped, the graph might present a different picture. In order to do this, let us look at the distribution of the computed weights (`weight`) in the dataframe.

In [80]:
# Disctibution of weights
print(df_ss.weight.describe())

count    198840.000000
mean          5.486316
std           5.322947
min           0.099538
25%           1.890026
50%           3.824369
75%           7.128170
max          71.351698
Name: weight, dtype: float64


Based on the numbers above, the mean weight of an edge in the graph is __5.486__,25% of the edges have weights less than __1.89__ and 50% of the edges have weights less than __3.82__. 

#### Remove edges with weights less than Q1 and draw

If we were to drop all edges that have weights less than the first quartile of 1.89, then we would be reducing the number of edges by 25%.

In [87]:
remove_edge_list = [] #define empty list
for edge in G_2.edges_iter(data = True):
    if edge[2]['weight'] < 1.89:
        # append edge to list
        remove_edge_list.append((edge[0],edge[1]))

print(len(remove_edge_list))
remove_edge_list

24852


[('Allein', 'Studen plamuk'),
 ('Allein', 'That Old Gang Of Mine'),
 ('Allein', 'Moon Going Down'),
 ('Allein', 'Murder One'),
 ('Allein', 'El Termometro'),
 ('Allein', 'Blue Yodel No. 1(T For Texas)'),
 ('Allein', 'Watermelon Man'),
 ('Allein', 'Ter\\xc3\\xa4slintu'),
 ('Allein', 'Straight Coats'),
 ('Allein', 'Transmission (Album Version)'),
 ('Allein', 'Des Helden Widersacher'),
 ('Allein', 'Kids Now'),
 ('Allein', 'Hear Me Talking To You'),
 ('Allein', 'Let It All Go'),
 ('Allein', 'Sentimental Journey'),
 ('Allein', 'Ooh Wee Baby'),
 ('Allein', 'We Do It For the Money_ OBVIOUSLY! (Demo)'),
 ('Allein', 'Here Comes The Big Parade'),
 ('Allein', 'Pus-Filled Carcass'),
 ('Allein', 'Big Woman'),
 ('Allein', 'Seni Unuturum'),
 ('Allein', 'Sou Vazo Diskola'),
 ('Allein', 'For Me And My Gal'),
 ('Allein', 'Another Djemba Djemba'),
 ('Allein', 'Ask The Angels'),
 ('Allein', 'Only If You Live There'),
 ('Allein', 'White Winter Hymnal'),
 ('Allein', "She's About A Mover"),
 ('Allein', 'Tides

There are 24,852 edges that will be removed from the 99,420 edges. 

In [88]:
o

74567


The graph now looks a little better but still there is one single cluster. 

#### Remove edges with weights less than Q2 and draw

If we were to drop all edges that have weights less than the first quartile of 3.82, then we would be reducing the number of edges by 50%.

In [94]:
remove_edge_list = [] #define empty list
for edge in G_2.edges_iter(data = True):
    if edge[2]['weight'] < 3.82:
        # append edge to list
        remove_edge_list.append((edge[0],edge[1]))

print(len(remove_edge_list))
remove_edge_list

24814


[('Allein', 'Speed Of Sound (Emergency Broadcast Album Version)'),
 ('Allein', 'Adios Pampa Mia'),
 ('Allein', 'But You Are'),
 ('Allein', 'Daughter Of Zion'),
 ('Allein', 'Burbullas A Grella'),
 ('Allein', 'Faithkeeper_ Part 1'),
 ('Allein', 'Your Love Takes Good Care Of Me'),
 ('Allein', 'Yesss Lord...'),
 ('Allein', 'C\\xc3\\xa9u do Brasil'),
 ('Allein', 'Rewind'),
 ('Allein', 'Skin Ticket (Live version) (Album Version)'),
 ('Allein', 'Street Sh***'),
 ('Allein', 'Love The Feeling - Original'),
 ('Allein', 'I Hate to Be the One to Say'),
 ('Allein', 'San Cristobal'),
 ('Allein', 'Your Touch Versus Death'),
 ('Allein', 'Nothing But The Best'),
 ('Allein', 'Posledny Buntar`'),
 ('Allein', 'Belle Isle Players'),
 ('Allein', 'Machuca'),
 ('Allein', '2StepN'),
 ('Allein', 'D\\xc3\\xa5 \\xc3\\xa4r det jul'),
 ('Allein', 'Banana Split Republic'),
 ('Allein', 'Contessa Entellina'),
 ('Allein', 'Yesterdays'),
 ('Allein', 'Flipside'),
 ('Allein', "She's All I Ever Had"),
 ('Allein', 'Gallows 

In [95]:
G_2_50 = G_2

# Remove the edges in remove_edge_list
G_2_50.remove_edges_from(remove_edge_list)
print(G_2_50.number_of_edges())

# Draw the network graph
nx.draw(G_2_50,
        pos=nx.spring_layout(G_2_50), # Or spectral_layout, random_layout, shell_layout , spring_layout     
        with_labels = False,
        node_size=100, 
        alpha=0.8, 
        node_color = [G_2_50.node[node]['color'] for node in G_2_50],#node_color_values['color'],
        font_size = 7,
        font_color = 'black',
        font_family = 'arial',
        edge_color = 'gray',
        width = [0.1*edge[2]['weight'] for edge in G_2_50.edges(data=True)])
        
pyplot.show()

49753


The nodes in the network graph spreads out even further. But still no clear multiple clusters.

#### Subgraph of the single track with all its neighbors

Let us look at the subgraph of the track titled _Thank You Mama_.

In [155]:
neighbors = nx.neighbors(G_2_50,'Thank You Mama')
neighbors.append('Thank You Mama')

G_2_sub = nx.subgraph(G_2_25,neighbors)

nx.draw(G_2_sub,
        pos=nx.spring_layout(G_2_sub), # Or spectral_layout, random_layout, shell_layout , spring_layout     
        with_labels = True,
        node_size=140, 
        alpha=0.8, 
        node_color = [G_2_sub.node[node]['color'] for node in G_2_sub],#node_color_values['color'],
        font_size = 11,
        font_color = 'black',
        font_family = 'arial',
        edge_color = 'gray',
        width = [0.2*edge[2]['weight'] for edge in G_2_sub.edges(data=True)])
        
pyplot.show()

The `ego_graph` module of `networkx` library also helps generate a sub graph, as shown below.

In [193]:
G_ego = nx.ego_graph(G_2_25, 'Thank You Mama', radius=1, center=True, undirected=False, distance=None)
# radius is the number defining the no. of edges within which the neighbors need to be identified

nx.draw(G_ego,
        with_labels = True,
        pos=nx.spring_layout(G_ego), # Or spectral_layout, random_layout, shell_layout , spring_layout     
        node_size=150, 
        alpha=0.8, 
        font_size = 11,
        font_color = 'black',
        font_family = 'arial',
        edge_color = 'gray',
        edge_labels = nx.get_edge_attributes(G_ego,'weight'),
        width = [0.1*edge[2]['weight'] for edge in G_ego.edges(data=True)]) # not working

#nx.draw_networkx_edge_labels(G_ego, nx.spring_layout(G_ego), labels = nx.get_edge_attributes(G_ego,'weight'), font_size = 1)

pyplot.show()

### Recommendation using Dijkstra's Algorithm

In this section, given a `track`, the 10 nearest songs are recommended using the __Dijkstra's__ algorithm. It is an algorithm for finding the shortest paths between nodes in a graph, which may represent, for example, road networks. More information on the algorithm can be found at https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm. 

For our recommendation, in addition to looking at the nearest directly connected neighbors, the nodes or the series of nodes that end up leading to the top 10 shortest paths will be looked at. The module `dijkstra_predecessor_and_distance` does this. Given an input node, it provides us with all the shortest path lengths and predecessors on these shortest paths. 

#### Recommendation for the track _Thank You Mama_

The Dijkstra's algorithm is used to recommend the 10 closest songs based on the shortest paths for the track __Thank You Mama__.

In [187]:
# For recommendations
short_path = nx.dijkstra_predecessor_and_distance(G_2_25, 'Thank You Mama', cutoff=0.2, weight='inv_weight')
# Returns two dictionaries, one containing the path traversed and the other containing the path distances.
print(type(short_path))
print(type(short_path[1]))
short_path

<class 'tuple'>
<class 'dict'>


({'By Faith I Can Touch Him Now': ['Thank You Mama'],
  'Encouraged': ['Thank You Mama'],
  "Job's Defiance": ['Thank You Mama'],
  'Never Alone': ['Thank You Mama'],
  'Packing Up': ['Thank You Mama'],
  'Pray On': ['Thank You Mama'],
  'So Many Reasons': ['Thank You Mama'],
  'Thank You Mama': [],
  'The Christmas Song  (LP Version)': ['Thank You Mama']},
 {'By Faith I Can Touch Him Now': 0.17351011923680854,
  'Encouraged': 0.1629578055774056,
  "Job's Defiance": 0.19294197292733542,
  'Never Alone': 0.1952223990227066,
  'Packing Up': 0.15996397932036563,
  'Pray On': 0.16767385166201704,
  'So Many Reasons': 0.19647779212008937,
  'Thank You Mama': 0,
  'The Christmas Song  (LP Version)': 0.18306110744959686})

In [197]:
# Get the nodes involved 
short_path_node_list = []
for key, value in short_path[0].items():
    temp = [key,value]
    short_path_node_list.append(temp)

short_path_node_list
#short_path_list[1]    
#short_path_list[1][1]

['Thank You Mama']

In [221]:
# Get the end nodes involved based on shortest path 
short_path_node_list = []
for key, value in short_path[1].items():
    temp = [key,value]
    short_path_node_list.append(temp)

#short_path_node_list
#short_path_node_list[1]    
#short_path_node_list[1][1]

# Convert the list into a dataframe
short_path_df = pd.DataFrame(np.nan, index=[i for i in range(0,len(short_path_node_list))], columns=['dist','end_node'])

for i in range(0,len(short_path_node_list)):
    short_path_df.end_node[i] = short_path_node_list[i][0]
    short_path_df.dist[i] = short_path_node_list[i][1]

short_path_df['rank'] = short_path_df['dist'].rank(axis=0, method='average', ascending=True, pct=False)
short_path_df

C:\Users\ganes_000\Anaconda3\envs\python3.5\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ganes_000\Anaconda3\envs\python3.5\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,dist,end_node,rank
0,0.173510,By Faith I Can Touch Him Now,5.0
1,0.183061,The Christmas Song (LP Version),6.0
2,0.195222,Never Alone,8.0
3,0.159964,Packing Up,2.0
4,0.000000,Thank You Mama,1.0
5,0.196478,So Many Reasons,9.0
6,0.167674,Pray On,4.0
7,0.162958,Encouraged,3.0
8,0.192942,Job's Defiance,7.0


In [243]:
# Pickle the graph object
path = "C:/Users/ganes_000/Documents/MSBA/Spring 2016/MA 755/Pickle Files"
nx.write_gpickle(G_2,path+"/Track_Track_Graph_500_edit.pkl")

In [244]:
# Read in the graph object
path = "C:/Users/ganes_000/Documents/MSBA/Spring 2016/MA 755/Pickle Files"
G =  nx.read_gpickle(path+"/Track_Track_Graph_500_edit.pkl")
G.number_of_edges()

99420

In [165]:
nx.single_source_shortest_path_length(G_2_25 ,source='Thank You Mama', cutoff=1)

{'By Faith I Can Touch Him Now': 1,
 'Encouraged': 1,
 "Job's Defiance": 1,
 'My Tribute (to God Be The Glory)': 1,
 'Never Alone': 1,
 'Packing Up': 1,
 'Pray On': 1,
 'Quiet Time': 1,
 'So Many Reasons': 1,
 'Thank You Mama': 0,
 'The Christmas Song  (LP Version)': 1}